<a href="https://colab.research.google.com/github/aida-am247/Deep-Learning/blob/master/E1/DL_E1_callback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/Alireza-Akhavan/SRU-deeplearning-workshop/master/dataset.py
!mkdir dataset
!wget https://github.com/Alireza-Akhavan/SRU-deeplearning-workshop/raw/master/dataset/Data_hoda_full.mat -P dataset

--2020-09-22 13:50:31--  https://raw.githubusercontent.com/Alireza-Akhavan/SRU-deeplearning-workshop/master/dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 929 [text/plain]
Saving to: ‘dataset.py’

dataset.py          100%[===================>]     929  --.-KB/s    in 0s      

2020-09-22 13:50:32 (47.1 MB/s) - ‘dataset.py’ saved [929/929]

--2020-09-22 13:50:32--  https://github.com/Alireza-Akhavan/SRU-deeplearning-workshop/raw/master/dataset/Data_hoda_full.mat
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Alireza-Akhavan/SRU-deeplearning-workshop/master/dataset/Data_hoda_full.mat [foll

In [2]:
import keras
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
import numpy as np
from dataset import load_hoda
import matplotlib.pyplot as plt

In [3]:
class my_callback(Callback):
  def on_epoch_end(self, epoch, logs={}):
    print(logs)
    if logs.get('accuracy') > 0.75:
      print("\nReached 75% accuracy so cancelling training!")
      self.model.stop_training = True
callback = my_callback()

In [4]:
np.random.seed(123)  # for reproducibility

# Load pre-shuffled HODA data into train and test sets
x_train_original, y_train_original, x_test_original, y_test_original = load_hoda(
                                                                        training_sample_size=3500,                                                                      test_sample_size=400,size=28)

# Preprocess input data
''' 3.1: input data in numpy array format'''
x_train = np.array(x_train_original)
x_test = np.array(x_test_original)
'''3.2 normalize our data values to the range [0, 1]'''
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Reshape to original image shape (n x 784)  ==> (n x 28 x 28 x 1)
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)


# Preprocess class labels
y_train = keras.utils.to_categorical(y_train_original, num_classes=10)
y_test = keras.utils.to_categorical(y_test_original, num_classes=10)


# test and validation set
x_val = x_test[:200]
x_test = x_test[200:]
y_val = y_test[:200]
y_test = y_test[200:]

In [5]:
# Define model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',
                        input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
# Fit model on training data
history = model.fit(x_train, y_train,
          epochs=200, batch_size=256, validation_data = (x_val, y_val), callbacks=[callback])

Epoch 1/200
14/14 [==============================] - 1s 40ms/step - loss: 2.0474 - accuracy: 0.3377 - val_loss: 1.3851 - val_accuracy: 0.7850
Epoch 2/200
14/14 [==============================] - 0s 13ms/step - loss: 1.2249 - accuracy: 0.6000 - val_loss: 0.5976 - val_accuracy: 0.8300
Epoch 3/200
14/14 [==============================] - 0s 13ms/step - loss: 0.8034 - accuracy: 0.7366 - val_loss: 0.4013 - val_accuracy: 0.8800
Epoch 4/200
11/14 [======================>.......] - ETA: 0s - loss: 0.6376 - accuracy: 0.7944{'loss': 0.6287561655044556, 'accuracy': 0.7991428375244141, 'val_loss': 0.32034769654273987, 'val_accuracy': 0.8799999952316284}

Reached 75% accuracy so cancelling training!
14/14 [==============================] - 0s 12ms/step - loss: 0.6288 - accuracy: 0.7991 - val_loss: 0.3203 - val_accuracy: 0.8800
